In [1]:
import pandas as pd
import numpy as np
import re

q-values and log2FC:

In [2]:
file_log2FC = '../../data/standardized_data/result_logfc_matrix_2020_08_27.csv'
df_lfc = pd.read_csv(file_log2FC)

file_qval = '../../data/standardized_data/result_qval_matrix_2020_08_27.csv'
df_qval = pd.read_csv(file_qval)

UniProt annotations:

In [3]:
file_annotation = '../../data/annotations/uniprot_annotation_scores.csv'
df_UNP = pd.read_csv(file_annotation)

CRISPRi: 

In [39]:
file_crispri = '/Users/ajinich/Documents/git_repos/tbredox/data/cloned_genes.xlsx'
df_crispri = pd.read_excel(file_crispri)
df_crispri.rename(columns = {'gene\nname':'gene_name'}, inplace=True)
df_crispri['gene_name'].fillna('', inplace=True)

df_crispri['Rv_ID'] = [''.join(gn.split('_')) for gn in df_crispri.gene_name]
df_crispri = df_crispri[df_crispri['BSL-3_Freezer'].values == True]

InterPro:

In [5]:
## get annotation according to UniProt:
file_UNP_1 = '../../data/annotations/uniprot_Mtb.tab'
df_UNP_1 = pd.read_csv(file_UNP_1, sep='\t')
df_UNP_1 = df_UNP_1[['Entry', 'Gene names']]

file_UNP_2 = '../../data/annotations/UP000001584.csv'
df_UNP_2 = pd.read_csv(file_UNP_2, sep='\t')
df_UNP_2 = df_UNP_2[['Entry', 'InterPro']]

df_UNP_12 = df_UNP_1.merge(df_UNP_2, how = 'inner', on = 'Entry')

rvid = [re.findall('Rv\d\d\d\d[a-zA-Z]?', gn)[0] for gn in df_UNP_12['Gene names']]
df_UNP_12['Rv_ID'] = rvid
df_UNP_12 = df_UNP_12[['Rv_ID', 'InterPro']]

file_IPR = '../../data/annotations/IPR_listing.xlsx'
df_IPR = pd.read_excel(file_IPR)

#### mouse in-vivo screens of interest:

In [6]:
screen_1 = 'dejesus_H37Rv_day32_vs_dejesus_H37Rv_day0'
screen_2 = 'zhang_wt_mouse_d10_vs_zhang_input_library'
screen_3 = 'zhang_wt_mouse_d45_vs_zhang_input_library'

#### Genes that are conditionally essential in 2 or 3 of the screens: 

In [7]:
q_val_th = 0.05
lfc_th = 1

In [8]:
rvids_screen_1 = df_qval[ (df_qval[screen_1].values <= q_val_th) ].Rv_ID.to_list()
rvids_screen_2 = df_qval[ (df_qval[screen_2].values <= q_val_th) ].Rv_ID.to_list()
rvids_screen_3 = df_qval[ (df_qval[screen_3].values <= q_val_th) ].Rv_ID.to_list()

In [9]:
rvids_screen_ANY = list(set(rvids_screen_1).union(set(rvids_screen_2)).union(set(rvids_screen_3)))
rvids_screen_ANY.sort()

In [10]:
rvids_screen_12 = list(set(rvids_screen_1).intersection(set(rvids_screen_2)))
rvids_screen_13 = list(set(rvids_screen_1).intersection(set(rvids_screen_3)))
rvids_screen_23 = list(set(rvids_screen_2).intersection(set(rvids_screen_3)))
rvids_screen_123 = list(set(rvids_screen_12).intersection(set(rvids_screen_3)))

In [11]:
rvids_screen_12.sort()
rvids_screen_13.sort()
rvids_screen_23.sort()
rvids_screen_123.sort()

#### Subset of these that are unknows:

In [123]:
rvids_screen_ANY_orphans = df_UNP[ (df_UNP.Rv_ID.isin(rvids_screen_ANY)) & (df_UNP.Annotation.values<=3)].Rv_ID.to_list()

Add UniProt annotation score:

In [124]:
df_orphan_iv_essentials = pd.DataFrame()
df_orphan_iv_essentials['Rv_ID'] = rvids_screen_ANY_orphans
df_orphan_iv_essentials = df_orphan_iv_essentials.merge(df_UNP, on = 'Rv_ID', how = 'left')

Annotate with which in vivo screens show a hit

In [125]:
df_orphan_iv_essentials['dejesus_d32'] = [df_qval[df_qval.Rv_ID.values == rvid][screen_1].values[0] <= q_val_th for rvid in df_orphan_iv_essentials['Rv_ID']]
df_orphan_iv_essentials['zhang_d10'] = [df_qval[df_qval.Rv_ID.values == rvid][screen_2].values[0] <= q_val_th for rvid in df_orphan_iv_essentials['Rv_ID']]
df_orphan_iv_essentials['zhang_d45'] = [df_qval[df_qval.Rv_ID.values == rvid][screen_3].values[0] <= q_val_th for rvid in df_orphan_iv_essentials['Rv_ID']]

Generate an in-vivo essentiality score: 

In [126]:
int1 = np.array([int(bit) for bit in df_orphan_iv_essentials.dejesus_d32.values])
int2 = np.array([int(bit) for bit in df_orphan_iv_essentials.zhang_d10.values])
int3 = np.array([int(bit) for bit in df_orphan_iv_essentials.zhang_d45.values])

score = int1 + int2 + int3
df_orphan_iv_essentials['score'] = score
df_orphan_iv_essentials.sort_values(by = ['score', 'Rv_ID'], ascending=False, inplace=True)

Add CRISPRi library information: 

In [127]:
df_orphan_iv_essentials['CRISPRi'] = df_orphan_iv_essentials.Rv_ID.isin(df_crispri.Rv_ID)
df_orphan_iv_essentials.reset_index(inplace=True, drop = True)

df_crispri_ID = df_crispri[['Rv_ID', 'ID']]
df_orphan_iv_essentials = df_orphan_iv_essentials.merge(df_crispri_ID, on = 'Rv_ID', how = 'left')

In [128]:
df_orphan_iv_essentials.drop_duplicates(subset=['Rv_ID'], inplace=True)

Add other Tn-Seq information:

In [130]:
screens_123 = [screen_1, screen_2, screen_3]

list_cols_string = []
list_cols_noGI_string = []

for rvid in df_orphan_iv_essentials.Rv_ID.values:
    cols_qval = [col for col in df_qval.columns[1:] if df_qval[df_qval.Rv_ID==rvid][col].values[0] <= q_val_th] 
    cols_lfc = [col for col in df_lfc.columns[1:] if np.abs(df_lfc[df_lfc.Rv_ID==rvid][col].values[0]) >= lfc_th] 
    cols_sig = list(set(cols_qval).intersection(set(cols_lfc)))
    # ignore carey screens
    cols_sig = [col for col in cols_sig if not('carey' in col)]
    # ignore mouse in vivo screens
    cols_sig = [col for col in cols_sig if not(col in screens_123)]
    cols_sig.sort()
    # ignore genetic interaction / gene-KO screens
    cols_sig_noGI = [col for col in cols_sig if  len(re.findall('Rv\d\d\d\d', col))==0 and not('KO' in col)]
    cols_string = ' | '.join(cols_sig)
    cols_noGI_string = ' | '.join(cols_sig_noGI)
    list_cols_string.append(cols_string)
    list_cols_noGI_string.append(cols_noGI_string)

In [131]:
df_orphan_iv_essentials['MtbTnDB'] = list_cols_string
df_orphan_iv_essentials['MtbTnDB_noGI'] = list_cols_noGI_string

Add InterPro protein domains information:

In [132]:
df_orphan_iv_essentials = df_orphan_iv_essentials.merge(df_UNP_12, on='Rv_ID', how = 'left')
df_orphan_iv_essentials['InterPro'] = df_orphan_iv_essentials['InterPro'].fillna('NA')

In [133]:
list_ip_domains_str = []
for rvid in df_orphan_iv_essentials.Rv_ID.values:
    ip = df_orphan_iv_essentials[df_orphan_iv_essentials.Rv_ID.values == rvid].InterPro.values[0]
    if not ip == 'NA':
        list_ip = [iipp.strip() for iipp in ip.split(';') if 'IPR' in iipp]
        ip_domains_str = ' | '.join(df_IPR[df_IPR.ENTRY_AC.isin(list_ip)].ENTRY_NAME.to_list())
    else:
        ip_domains_str = 'NA'
    list_ip_domains_str.append(ip_domains_str)
df_orphan_iv_essentials['InterPro_Name'] = list_ip_domains_str

Final clean-up:

In [134]:
df_orphan_iv_essentials.drop_duplicates(subset='Rv_ID', inplace=True)
df_orphan_iv_essentials.replace('', 'NONE', inplace=True)

Write to file: 

In [135]:
# ALL 
file_out = '../../data/annotations/orphan_invivo_essentials.xlsx'
df_orphan_iv_essentials.to_excel(file_out, index = False)

# Tier I: Essential in 2 invivo screens, and in our CRISPRi library:
df_orphan_iv_essentials_TierI = df_orphan_iv_essentials[ (df_orphan_iv_essentials.score.values>=2) & 
                                                        (df_orphan_iv_essentials.dejesus_d32.values) & 
                                                        (df_orphan_iv_essentials.CRISPRi.values)]
file_out = '../../data/annotations/orphan_invivo_essentials_TierI.xlsx'
df_orphan_iv_essentials_TierI.to_excel(file_out, index = False)

# Tier II: In our CRISPRi library:
df_orphan_iv_essentials_TierII = df_orphan_iv_essentials[(df_orphan_iv_essentials.CRISPRi.values)]
file_out = '../../data/annotations/orphan_invivo_essentials_TierII.xlsx'
df_orphan_iv_essentials_TierII.to_excel(file_out, index = False)

# Tier III: Essential in the Minato et al. in vitro (minimal media) screen:
df_orphan_iv_essentials_TierIII = df_orphan_iv_essentials[df_orphan_iv_essentials.MtbTnDB_noGI.str.contains('minato')]
file_out = '../../data/annotations/orphan_invivo_essentials_TierIII.xlsx'
df_orphan_iv_essentials_TierIII.to_excel(file_out, index = False)

### Other ways to do this: 

In [ ]:
rvids_screen_1_orphans = df_UNP[ (df_UNP.Rv_ID.isin(rvids_screen_1)) & (df_UNP.Annotation.values<=3)].Rv_ID.to_list()

In [35]:
rvids_screen_12_orphans = df_UNP[ (df_UNP.Rv_ID.isin(rvids_screen_12)) & (df_UNP.Annotation.values<=3)].Rv_ID.to_list()
rvids_screen_13_orphans = df_UNP[ (df_UNP.Rv_ID.isin(rvids_screen_13)) & (df_UNP.Annotation.values<=3)].Rv_ID.to_list()
rvids_screen_23_orphans = df_UNP[ (df_UNP.Rv_ID.isin(rvids_screen_23)) & (df_UNP.Annotation.values<=3)].Rv_ID.to_list()
rvids_screen_123_orphans = df_UNP[ (df_UNP.Rv_ID.isin(rvids_screen_123)) & (df_UNP.Annotation.values<=3)].Rv_ID.to_list()

In [72]:
len(rvids_screen_12_orphans), len(rvids_screen_13_orphans), len(rvids_screen_23_orphans), len(rvids_screen_123_orphans)

(19, 83, 51, 14)

In [73]:
rvids_screen_123_orphans

['Rv0238',
 'Rv0242c',
 'Rv0988',
 'Rv1078',
 'Rv1159',
 'Rv1432',
 'Rv1829',
 'Rv2198c',
 'Rv2379c',
 'Rv2381c',
 'Rv2553c',
 'Rv2563',
 'Rv3267',
 'Rv3682']

#### Subset of these in our CRISPRi database:

In [65]:
rvids_screen_12_orphans_CRISPRi = df_crispri[df_crispri.Rv_ID.isin(rvids_screen_12_orphans)].Rv_ID.to_list()
rvids_screen_12_orphans_CRISPRi.sort()

rvids_screen_13_orphans_CRISPRi = df_crispri[df_crispri.Rv_ID.isin(rvids_screen_13_orphans)].Rv_ID.to_list()
rvids_screen_13_orphans_CRISPRi.sort()

rvids_screen_23_orphans_CRISPRi = df_crispri[df_crispri.Rv_ID.isin(rvids_screen_23_orphans)].Rv_ID.to_list()
rvids_screen_23_orphans_CRISPRi.sort()

rvids_screen_123_orphans_CRISPRi = df_crispri[df_crispri.Rv_ID.isin(rvids_screen_123_orphans)].Rv_ID.to_list()
rvids_screen_123_orphans_CRISPRi.sort()

In [67]:
rvids_screen_12_orphans_CRISPRi

['Rv1432', 'Rv2500c']

In [68]:
rvids_screen_13_orphans_CRISPRi

['Rv0244c', 'Rv1272c', 'Rv1405c', 'Rv1432', 'Rv3283', 'Rv3400', 'Rv3502c']

In [69]:
rvids_screen_23_orphans_CRISPRi

['Rv1193', 'Rv1432', 'Rv3032']

In [66]:
rvids_screen_123_orphans_CRISPRi

['Rv1432']